In [1]:
import sys
from pathlib import Path

# Get the root directory of the project
project_root = Path("/home/lxz/scmamba/KCellFM_tutorial/spatial_transcriptomics").parent.parent
# project_root = Path(__file__).parent.parent
sys.path.append(str(project_root))

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,classification_report
from models.model import MambaModel
from models.gene_tokenizer import GeneVocab
import pickle

In [9]:
# Configure parameters (consistent with training)
batch_size = 96
embsize = 512
nhead = 8
d_hid = 512
nlayers = 6
dropout = 0.1
pad_token = "<pad>"
max_seq_len = 50
input_emb_style = "continuous"
cell_emb_style = "cls"
mask_value = -1
pad_value = -2

In [10]:
# Load vocabulary and gene mapping
vocab = GeneVocab.from_file("/home/lxz/scmamba/vocab.json")
ntokens = len(vocab)

In [11]:
# Gene Mapping
gene_to_id = {
    "MUC2": 17183, "SOX9": 32052, "MUC1": 17175, "CD31": 19330, "Synapto": 32742,
    "CD49f": 12272, "CD15": 9687, "CHGA": 4894, "CDX2": 4568, "ITLN1": 12308,
    "CD4": 4380, "CD127": 12051, "Vimentin": 35192, "HLADR": 11044, "CD8": 4412,
    "CD11c": 12283, "CD44": 4383, "CD16": 9286, "BCL2": 3080, "CD123": 36627,
    "CD38": 4376, "CD90": 33320, "aSMA": 1391, "CD21": 5523, "NKG2D": 12911,
    "CD66": 4589, "CD57": 2956, "CD206": 16892, "CD68": 4397, "CD34": 4373,
    "aDef5": 7546, "CD7": 4399, "CD36": 4374, "CD138": 30796, "Cytokeratin": 41736,
    "CK7": 12989, "CD117": 12801, "CD19": 4335, "Podoplanin": 19298, "CD45": 20664,
    "CD56": 17477, "CD69": 4398, "Ki67": 16711, "CD49a": 12264, "CD163": 4329,
    "CD161": 12901
}

# Cell type mapping (must be exactly the same as during training)
CL_cross ={
    "CD4T": 0,
    "CD7_Immune": 1,
    "CD8T": 2,
    "DC": 3,
    "Endothelial": 4,
    "Enterocyte_ITLN1p": 5,
    "Goblet": 6,
    "ICC": 7,
    "Lymphatic": 8,
    "Macrophage": 9,
    "Nerve": 10,
    "Neuroendocrine": 11,
    "Neutrophil": 12,
    "Paneth": 13,
    "Plasma": 14,
    "Stroma": 15,
    "TA": 16
}

CL_intra = {
    "CD4T": 0,
    "CD8T": 1,
    "DC": 2,
    "Endothelial": 3,
    "Enterocyte_ITLN1p": 4,
    "Goblet": 5,
    "ICC": 6,
    "Lymphatic": 7,
    "Macrophage": 8,
    "Nerve": 9,
    "Neuroendocrine": 10,
    "SmoothMuscle": 11,
    "Stroma": 12,
    "TA": 13
}


SB_cross = {
    "B": 0,
    "CD4T": 1,
    "CD7_Immune": 2,
    "CD8T": 3,
    "DC": 4,
    "Endothelial": 5,
    "Enterocyte_ITLN1p": 6,
    "Goblet": 7,
    "ICC": 8,
    "Lymphatic": 9,
    "Macrophage": 10,
    "Nerve": 11,
    "Neuroendocrine": 12,
    "Neutrophil": 13,
    "Plasma": 14,
    "Stroma": 15,
    "TA": 16
}

SB_intra = {
    "CD4T": 0,
    "CD7_Immune": 1,
    "CD8T": 2,
    "DC": 3,
    "Endothelial": 4,
    "Goblet": 5,
    "ICC": 6,
    "Lymphatic": 7,
    "Macrophage": 8,
    "Nerve": 9,
    "Neuroendocrine": 10,
    "Neutrophil": 11,
    "Plasma": 12,
    "Stroma": 13,
    "TA": 14
}

celltype_to_id = CL_cross  # CL_cross, CL_intra, SB_cross, SB_intra

In [12]:
id_to_celltype = {v: k for k, v in celltype_to_id.items()}
class_num = len(celltype_to_id)

In [13]:
# Test dataset class (same as training)
class TestDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.genes = [col for col in self.data.columns if col in gene_to_id]
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        vocab_ids = []
        expr_values = []
        for gene in self.genes:
            vocab_ids.append(gene_to_id[gene])
            expr_values.append(row[gene])
        
        vocab_ids = [60695] + vocab_ids  # <cls> token
        expr_values = [0.0] + expr_values
        
        padding_length = max_seq_len - len(vocab_ids)
        if padding_length > 0:
            vocab_ids = vocab_ids + [60694] * padding_length  # <pad> token
            expr_values = expr_values + [pad_value] * padding_length
        
        padding_mask = [False] * len(vocab_ids)
        for i in range(len(vocab_ids)):
            if vocab_ids[i] == 60694:
                padding_mask[i] = True
        
        cell_type = row['cell_type_A']
        cell_type_id = celltype_to_id.get(cell_type, -1)  # -1 represents an unknown type
        
        return {
            'src': torch.tensor(vocab_ids, dtype=torch.long),
            'values': torch.tensor(expr_values, dtype=torch.float),
            'padding_mask': torch.tensor(padding_mask, dtype=torch.bool),
            'celltype': torch.tensor(cell_type_id, dtype=torch.long),
            'coordinates': torch.tensor([row['x'], row['y']], dtype=torch.float)
        }

In [14]:
def evaluate():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Initialize model
    model = MambaModel(
        ntokens, embsize, nhead, d_hid, nlayers,
        dropout=dropout, pad_token=pad_token,
        pad_value=pad_value, input_emb_style=input_emb_style,
        cell_emb_style=cell_emb_style, class_num=class_num
    ).to(device)
    
    # Load the weights of the trained model
    model_path = "/home/lxz/scmamba/model_state/spatial_classifier_CL_cross_best.pth"  
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    # Load test data
    test_dataset = TestDataset("/home/lxz/scmamba/空转/Hubmap_CL_cross/test.csv")  
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4
    )
    
    # Store predicted results
    all_preds = []
    all_labels = []
    all_cell_embs = []
    
    with torch.no_grad():
        for batch in test_loader:
            src = batch['src'].to(device)
            values = batch['values'].to(device)
            padding_mask = batch['padding_mask'].to(device)
            cell_types = batch['celltype'].to(device)
            
            # forward propagation
            model_output = model(
                src=src,
                values=values,
                src_key_padding_mask=padding_mask
            )
            
            # Obtain predicted categories
            preds = torch.argmax(model_output["cls_output"], dim=1)
            cell_embs = model_output["cell_emb"].cpu().numpy()
            
            # Collect results (filter unknown types)
            valid_mask = cell_types != -1
            all_preds.extend(preds[valid_mask].cpu().numpy())
            all_labels.extend(cell_types[valid_mask].cpu().numpy())
            all_cell_embs.append(cell_embs[valid_mask.cpu().numpy()])  # Only retain embeddings of valid samples
    
    # Splicing the embeddings of all batches together
    all_cell_embs = np.concatenate(all_cell_embs, axis=0)  # Shape [N_valid, 512]
    
    # Save data
    np.save("/home/lxz/scmamba/空转/Hubmap_CL_cross/cell_embeddings_test.npy", all_cell_embs)  
    
    # Save labels and preds to a. pkl file
    with open("/home/lxz/scmamba/空转/Hubmap_CL_cross/labels_preds_test.pkl", "wb") as f:  
        pickle.dump({
            "labels": all_labels,  # true label
            "preds": all_preds     # prediction label
        }, f)
    
    # compute accuracy
    if len(all_preds) > 0:
        acc = accuracy_score(all_labels, all_preds)
        print(f"\nTest Accuracy: {acc:.4f}")
        
        # Print classification report
        print("\nDetailed Classification Report:")
        print(classification_report(
            all_labels, all_preds, 
            target_names=[id_to_celltype[i] for i in range(class_num)],
            labels=list(range(len(celltype_to_id))),
            digits=4
        ))
    else:
        print("No valid samples for evaluation")

In [15]:
if __name__ == "__main__":
    evaluate()


Test Accuracy: 0.8047

Detailed Classification Report:
                   precision    recall  f1-score   support

             CD4T     0.8030    0.7471    0.7740      5259
       CD7_Immune     0.4796    0.6092    0.5367       174
             CD8T     0.6738    0.9411    0.7854      4127
               DC     0.7933    0.7981    0.7957      1342
      Endothelial     0.9498    0.9780    0.9637      6504
Enterocyte_ITLN1p     0.5872    0.7971    0.6762       207
           Goblet     0.5878    0.9448    0.7248      7356
              ICC     0.8713    0.9077    0.8891      1029
        Lymphatic     0.8567    0.5741    0.6875      2552
       Macrophage     0.8964    0.9117    0.9040      9372
            Nerve     0.6354    0.9471    0.7606      3329
   Neuroendocrine     0.9150    0.9340    0.9244       530
       Neutrophil     0.7912    0.9000    0.8421       400
           Paneth     0.0000    0.0000    0.0000         0
           Plasma     0.9419    0.8028    0.8668     11425

/home/lxz/tools/anaconda3/envs/scgpt/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lxz/tools/anaconda3/envs/scgpt/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lxz/tools/anaconda3/envs/scgpt/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu